Antaget dataformat:

Local Time | Open | High | Low | Close | Volume | Name

Dollars-bar funktionen skal indeholde:

* Tilføj ny 'Mean' kolonne, som er udregnes
$$
\begin{align*}
    \text{Mean_HL} = \frac{\text{High} + \text{Low}}{2}
\end{align*}
$$

* Tilføj variablen 'Dollarbar_cap', som er $\text{DollarBar_cap} = max(\text{High}) \cdot max(\text{volume})$ skal rundes op til nærmeste million

* Tilføj variablen 'Total_transaction' som er $\text{Total_transaction}=\text{Rest_transaction}$ udenfor loop, og $\text{Total_transaction}=\text{Total_transaction}+\text{Volume}$ inde i loop.

* Tilføj variablen 'Rest_volume', som er $\text{Rest_transaction} = \text{Total_transaction} - \text{Dollarbar_cap}$, med start værdi 0, og kun når 'Total_transaction' > DollarBar_cap

* Tilføj vairablen 'Mean_OC', som er $\text{Mean_OC} = \frac{\text{Open} + \text{Close}}{2}$

* Tilføj variablen 'High_list' and 'Low_list', som skal append 'High' og 'Low' og når Dollarbar_cap er nået, så tage max og min af listen




start den førstes variable 'open' til 'open' og sæt 'close til den sidste rækkes open og close middelværdi (Mean_OC[1])
start den næste 'open' med Mean_OC[1] og close med Mean_OC[2]




Gem variable

* 'Dato' skal være slut Dato
* 'Open' fra 'Open' i starten ellers 'Mean_OC'
* 'High' og 'Low' fra HL_list, hvor der er taget min og max
* 'Close' fra 'Mean_OC'
* 'Type' fra 'type' i for loop variable 
* 'Volume' som er Dollarbar_cap


Funktionen virker nok ikke helt optimalt i det special tilfælde, hvor vi skal lave en dollarsbar på den sidste observation i aktie data'et. Som funktionen er nu, så vil den sidste dollarsbar være "rest", og her kan vi lægge under Dollarsbar_cap, men også over, hvilket jeg ikke er sikker på om det er okay.

Index driller, men ved ikke om det er vigtigt.

In [88]:
import matplotlib
import pandas as pd
import numpy as np
import time
import os as os


start=time.time()  # Start the time


def dollarsbar(df):
    if isinstance(df, pd.DataFrame):
        
        ## Create empty dataframe
        df_Dollarsbar = pd.DataFrame({'Local time' : [], 'Open' : [], 'Low' : [], 'High' : [], 
                                      'Close' : [], 'Volume' : [], 'Monetary_Volume' : [], 'Name' : []})
        
        for name in pd.unique(df['Name']):
    
            df_type = df[df['Name'] == name]
            df_type = df_type.reset_index(drop=True)
            Rest_transaction = 0
            Total_transaction = 0
    
            ## Empty lists
            Time_list = []
            Open_list = []
            Low_list = []
            High_list = []
            Close_list = []
            Volume_list = []
            HL_list = []
    
            ## support vairables
            Volume_help_list = []
            iter_since_dollarsbar = 0
    
            ## Dollarsbar cap
            max_High = np.max(df_type['High'])
            max_Volume = np.max(df_type['Volume'])
            Dollarsbar_cap = max_High * max_Volume
    
            ## First value in Open
            Open_list.append(df_type['Open'][0])
    
            for n in np.arange(df_type.shape[0]):        
            
                Time = df_type['Local time'][n]
                Open = df_type['Open'][n]
                High = df_type['High'][n]
                Low = df_type['Low'][n]
                Close = df_type['Close'][n]
                Volume = df_type['Volume'][n]
        
                Mean_HL = np.mean([High, Low])
                HL_list.append(High)
                HL_list.append(Low)
        
                Total_transaction = Total_transaction + Mean_HL*Volume 
                Mean_OC = np.mean([Open, Close])
        
                Volume_help_list.append(Volume)
        
                iter_since_dollarsbar += 1
            
                if n % 10000 == 0:
                    print(n)
        
                if Dollarsbar_cap < Total_transaction:   #and df_type.shape[0] > n
            
                    Rest_transaction = Total_transaction - Dollarsbar_cap
            
                    max_High = np.max(HL_list)
                    min_Low = np.min(HL_list)
            
                    p = (Dollarsbar_cap - Total_transaction + Mean_HL*Volume)/(Mean_HL*Volume) ## andel af volume som skal med i denne dollarsbar
                    Volume_help_list[-1] = Volume*p
                    Sum_volume = np.sum(Volume_help_list)
            
                    ## Saving dollarbar values
                    Time_list.append(Time)
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Close_list.append(Mean_OC)
                    Volume_list.append(Sum_volume)
                    Open_list.append(Mean_OC)
                    
                    HL_list = []
            
                    Volume_help_list = []
                    Volume_help_list.append(Volume*(1-p)) ## andel af volume som skal med i næste dollarsbar
                    
                    Total_transaction = Rest_transaction
            
                    iter_since_dollarsbar = 0
            
                if df_type.shape[0] == n+1 and iter_since_dollarsbar == 0:
                
                    Sum_volume = Volume * (1-p)
                
                    Close_list.append(Close)
                    Time_list.append(Time)  ## Time bliver det samme, hvilket ik er godt
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Volume_list.append(Sum_volume)
            
                    df_dummy = pd.DataFrame({'Local time' : Time_list, 'Open' : Open_list, 'Low' : Low_list, 'High' : High_list, 
                                     'Close' : Close_list, 'Volume' : Volume_list, 
                                     'Monetary_Volume' : Dollarsbar_cap, 'Name' : name})
                    df_Dollarsbar = pd.concat([df_Dollarsbar,df_dummy.iloc[:-1,:]])
            
                if df_type.shape[0] == n+1 and iter_since_dollarsbar > 0:
            
                    max_High = np.max(HL_list)
                    min_Low = np.min(HL_list)
            
                    Sum_volume = np.sum(Volume_help_list)
            
                    ## Saving dollarbar values
                    Time_list.append(Time)
                    Low_list.append(min_Low)
                    High_list.append(max_High)
                    Close_list.append(Mean_HL)
                    Volume_list.append(Sum_volume)
            
                    df_dummy = pd.DataFrame({'Local time' : Time_list, 'Open' : Open_list, 'Low' : Low_list, 'High' : High_list, 
                                     'Close' : Close_list, 'Volume' : Volume_list, 
                                     'Monetary_Volume' : Dollarsbar_cap, 'Name' : name})
                    df_dummy['Monetary_Volume'].iloc[-1] = Total_transaction
                    df_Dollarsbar = pd.concat([df_Dollarsbar,df_dummy])
        
        return df_Dollarsbar
            
    else:
        print('Input need to be a Pandas DataFrame')
            



# Print the time usage:
end = time.time()  # Stop the time
print(f"Time used by LightGBM: {(end-start):.1f} seconds")

Time used by LightGBM: 0.0 seconds


In [ ]:
## Read data

#os.chdir("DOWNLOADSMAPPEN")
df_DAX = pd.read_csv('DAX.txt')


## Running the function 
a = dollarsbar(df_DAX)

0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000
260000
270000
280000
290000
300000
310000
320000
330000
340000
350000
360000
370000
380000
390000
400000
410000
420000
430000


In [86]:
a

,Local time,Open,Low,High,Close,Volume,Name
0,02.01.2018 08:07:00.000 GMT+0100,12926.2790,12925.280,12943.279,12940.0295,1.271315e+07,DAX
1,02.01.2018 08:11:00.000 GMT+0100,12940.0295,12925.280,12946.780,12943.6295,1.140395e+06,DAX
2,02.01.2018 08:17:00.000 GMT+0100,12943.6295,12925.280,12946.780,12936.6295,-5.375791e+06,DAX
3,02.01.2018 08:22:00.000 GMT+0100,12936.6295,12925.280,12946.780,12940.3795,1.891735e+06,DAX
4,02.01.2018 08:27:00.000 GMT+0100,12940.3795,12925.280,12946.780,12937.5295,2.575642e+07,DAX
...,...,...,...,...,...,...,...
229,03.01.2018 10:38:00.000 GMT+0100,12898.7395,12746.979,12949.979,12902.1295,3.528964e+06,DAX
230,03.01.2018 10:42:00.000 GMT+0100,12902.1295,12746.979,12949.979,12910.9790,1.124538e+06,DAX
231,03.01.2018 10:46:00.000 GMT+0100,12910.9790,12746.979,12949.979,12910.8800,1.430970e+06,DAX
232,03.01.2018 10:50:00.000 GMT+0100,12910.8800,12746.979,12949.979,12919.6290,2.356107e+06,DAX


In [87]:
df_DAX.describe()

,Unnamed: 0,Open,High,Low,Close,Volume
count,1.394003e+06,1.394003e+06,1.394003e+06,1.394003e+06,1.394003e+06,1.394003e+06
mean,2.589977e+05,1.313958e+04,1.314198e+04,1.313705e+04,1.313950e+04,5.395194e+04
std,1.737638e+05,1.595417e+03,1.595096e+03,1.595675e+03,1.595384e+03,1.354461e+05
min,0.000000e+00,7.965880e+03,7.980380e+03,7.965880e+03,7.972860e+03,1.000000e+00
25%,1.161665e+05,1.209917e+04,1.210121e+04,1.209688e+04,1.209898e+04,8.450000e+03
50%,2.323330e+05,1.286598e+04,1.286817e+04,1.286360e+04,1.286591e+04,2.631000e+04
75%,3.814895e+05,1.425590e+04,1.426117e+04,1.425105e+04,1.425579e+04,5.743000e+04
max,6.665880e+05,1.629770e+04,1.629770e+04,1.629610e+04,1.629710e+04,5.697120e+06
